<a href="https://colab.research.google.com/github/Yuanyyz1/test_codex/blob/main/Medical_translation_openrouter_dspy_updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Medical translation overview
**1. Error Insertion**
- Generating medical conversation. This is one way from doctor -> patients. (en_1)
- Translating medical coversation into Chinese (cn_1)
- Inserting errors in Chinese (cn_2)
- Translatingt back the errored Chinese into English (en_2)

<br>

*Input*

<br>

1. Original English Text: a doctor–patient conversation in English.
2. Original Chinese Translation: the accurate, error-free Chinese translation of the conversation.
3. Error table: a list of possible translation errors (e.g. Units & Dosage, Negation, Terminology, etc.).


<br>

*Error insertion example:*

```
{'errors': 'Units & Dosage',
 'errors_domain': 'Errors in Medical Conversations',
 'errors_explain': 'The frequency of the medication administration was incorrectly translated, which could lead to improper treatment scheduling}
```  
  
*Inserted errored sentences and orginal sentence:*

```
 {'en_2': "Okay, now that you've completed your surgery and chemotherapy, we're going to start you on a medication called Herceptin, also known as trastuzumab. This medicine is specifically for your HER2-positive breast cancer, and its main job is to help reduce the chance of the cancer coming back. You'll receive it as an intravenous infusion, meaning it will be given through a drip into a vein, usually in your arm, every two weeks for about one year.",
 'cn_2': '好的，既然您已经完成了手术和化疗，我们将为您开始使用一种名为赫赛汀，通用名为曲妥珠单抗的药物。这种药物专门用于治疗您的HER2阳性乳腺癌，其主要作用是帮助降低癌症复发的风险。您将通过静脉输注的方式接受此药，这意味着药物将通过点滴进入您的静脉，通常是在您的手臂上，每两周一次，持续大约一年。',
 'en_1': '"Okay, now that you\'ve completed your surgery and chemotherapy, we\'re going to start you on a medication called Herceptin, also known as trastuzumab. This medicine is specifically for your HER2-positive breast cancer, and its main job is to help reduce the chance of the cancer coming back. You\'ll receive it as an intravenous infusion, meaning it will be given through a drip into a vein, usually in your arm, every three weeks for about one year.",',
 'cn_1': '"好的，既然您已经完成了手术和化疗，我们将为您开始使用一种名为赫赛汀，通用名为曲妥珠单抗的药物。这种药物专门用于治疗您的HER2阳性乳腺癌，其主要作用是帮助降低癌症复发的风险。您将通过静脉输注的方式接受此药，这意味着药物将通过点滴进入您的静脉，通常是在您的手臂上，每三周一次，持续大约一年。",'}
```

<br><br>

**2. Error detection (back-translation approach, AI1)**
- Generating response regarding the error
- Detecting the error

<br>

*Input*

<br>

1. Original English Text (`en_1`): a sentence from a doctor–patient conversation.
2. Chinese Translation (`cn_2`): the translated version of the English text.
3. Error Table: a list of possible translation error types.
4. Criteria for Serious Clinical Consequences.
5. Output Schema: a strict JSON structure to follow.

<br>

*Example of output from AI1:*
```
{'errors_detected': 'Units & Dosage & Temporal',
 'errors_detected_domain': 'Errors in Medical Conversations',
 'clinical_consequences_type': 'diagnostic or therapeutic intervention',
 'clinical_consequences_explain': 'The error in the frequency of administration (every two weeks vs. every three weeks) could lead to incorrect dosing schedule, affecting the effectiveness of the treatment.',
 'clinical_consequences_flag': 'Y'}
```

<br><br>


**3. Error detection validation**
- Checking whether if the resoponses in 2 is correct (whether if AI1 output is correct).

  
  a. it can detect whether if 2 is right where: 1. there is no error insertion and 2. there is error insertion.

  b. it can detect whether if 2 is wrong where 1. there is no error insertion and 2. there is no error insertion.

<br>

*Input*

<br>

1. Original_input: a sentence from a doctor–patient conversation (en_1), and the translated Chinese (cn_2).
2. AI generated output to evaluate the translation (errors_detected, errors_detected_domain, clinical_consequences_type, clinical_consequences_explain, clinical_consequences_flag)
3. An error table.
4. Criteria for Serious Clinical Consequences
5. Output Schema: a strict JSON structure to follow.
<br>


*Example of output when AI1 detect the wrong error:*

```
  {'ai1_correct': 'N',
 'ai1_correct_why': "The AI incorrectly identified the error as Phonetic Confusion. The actual error is in the frequency of administration. The original states 'every three weeks' while the translation states '每一周一次' (every week once), which is a Units & Dosage error.",
 'errors_detected': 'Units & Dosage',
 'errors_detected_domain': 'Errors in Medical Conversations',
 'clinical_consequences_type': 'diagnostic or therapeutic intervention',
 'clinical_consequences_explain': 'The error in dosage frequency could lead to incorrect administration of the medication, affecting the treatment plan.',
 'clinical_consequences_flag': 'Y'}

```

<br><br>


## Quick note
1. from code to text: Ctrl + mm
2. from text to code: Ctrl + MY


# File management


1. Checking files in the environment
```
files = os.listdir()
print(files)
```

2. Remove files (example)
- By group
```
files_to_delete = ['en_1 (1).txt', 'en_1 (2).txt']
for f in files_to_delete:
    if os.path.exists(f):
        os.remove(f)
        print(f"Deleted: {f}")
    else:
        print(f"Not found: {f}")
```

- Alone
```
os.remove("en_1.txt")
```

3. Remove object in the environment
```
del en_1
```

4. Save file
```
# save it to google drive
with open("translation_output_0.json", "w", encoding="utf-8") as f:
    json.dump(test_JSON , f, ensure_ascii=False, indent=2)
```
```
# save it to locally
from google.colab import files
files.download("translation_output_0.json")
```

5. Load file
```
# load it from google drive
file_path = "/content/translation_output_0.json"
with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)
```
```
# load it from locally (this will upload it to drive)
from google.colab import files
uploaded = files.upload() # will open a window
```

# Dspy tricks

1. the way you know what model is used to get this output
```
# turn on the usage
dspy.settings.configure(lm = lm,track_usage=True)

# use the output
validate_error_details.get_lm_usage() -> not working right now
lm.model -> this is working
```

2. they way you know the global model
```
dspy.settings.lm.model (haven't check this one)
```

# Set up the environment

## Import packages

In [ ]:
# Google Generative AI Python SDK
#!pip -q install --upgrade openai
# test updates
!pip install litellm
!pip install dspy
# What is SDK:
# Python functions to interact with Google Gemini models
# Ready-to-use methods like generate_content() or configure()
# Behind-the-scenes code that handles API requests, responses, errors
# Tools to make it easier and faster to work with Google's Generative AI

In [ ]:
# import packages

import dspy
import os
import pandas as pd
import time
import json
import re
from openai import OpenAI,APITimeoutError, APIConnectionError, APIStatusError
from typing import Optional, Dict, Any, List, Union
from google.colab import userdata # use the secrets
from litellm import completion
from collections import Counter
api_key = userdata.get('corey')
os.environ["OPENROUTER_API_KEY"] = api_key

# openrouter completion doc: https://openrouter.ai/docs/api-reference/completion


## Functions

In [ ]:
# 1. remove wordcounts less than 5
def word_count(text):
    # If text is missing or None → return 0
    if text is None:
        return 0
    # If text is a list → join it into a string
    if isinstance(text, list):
        text = ' '.join(text)
    # If text is not a string at this point → convert to string
    if not isinstance(text, str):
        text = str(text)
    # Split into words
    words = text.split()

    return len(words)

In [ ]:
# 2. print the loop number and model
# e.g. print_results(error_details, turn_cal, Pharmacist_results)
def print_results(initial_results, nloop, nresults, context = False):
  if context is True:
      return print("current model:",model_name,  # should match turn_cal
      "\nloop times:", nloop, # number of turns processed
      "\ntotal generated results (rows):",len(nresults))  # printout the model is used in the loop

  return print("current model:",list(initial_results.get_lm_usage())[0],  # should match turn_cal
      "\nloop times:", nloop, # number of turns processed
      "\ntotal generated results (rows):",len(nresults))  # printout the model is used in the loop

# 2.1 if previous function not working, use this one, but it will only print out the current global model
# def print_results(initial_results, nloop, nresults):
#   return print("current model:",lm.model,  # should match turn_cal # in case the last time it was not working
#       "\nloop times:", nloop, # number of turns processed
#       "\ntotal generated results (rows):",len(nresults))  # printout the model is used in the loop

In [ ]:
# 3. description of error types or any other keys in the dict
def counter_summary(data, fields):
    summary = Counter(
        item.get(field)
        for item in data
        for field in fields
        if item.get(field)  # skip None or empty values
    )
    return summary

In [ ]:
# clean the description of error types or any other keys in the dict
# wraped with previous function
# e.g. counter_summary_clean(Pharmacist_results_JSON, ["errors_1", "errors_2", "errors_3"], True)
def counter_summary_clean(data, fields, clean = True):
    # Build the raw counter
    counter = counter_summary(data, fields)

    # If flag is NOT True → return raw counter as-is
    if not clean:
      print("you are reading the raw Counter() results below:")
      return sorted(counter.items(), key=lambda x: x[1], reverse=True)

    # Otherwise continue with cleaning
    new = {}
    for key, value in counter.items():
        clean_key = re.sub(r"\s*\([^)]*\)", "", key)   # remove (xxx)
        new[clean_key] = new.get(clean_key, 0) + value

    # Sort the cleaned results
    sorted_new = sorted(new.items(), key=lambda x: x[1], reverse=True)

    # Summary info
    print(
        "summary of records:", sum(counter.values()),
        "\nsummary of cleaned records:", sum(new.values())
    )

    return sorted_new



### Conversation with detected errors

In [ ]:
# # 2. turn it into JSON



# # conversations= pd.read_excel("Pharmacist_Patients_total.xlsx")
# # conversations_JSON = conversations.to_json(orient="records", force_ascii=False)
# # conversations_JSON = json.loads(conversations_JSON)

# conversations_JSON_detect= pd.read_excel("conversations_JSON_detect_results_df.xlsx")
# conversations_JSON_detect_results = conversations_JSON_detect.to_json(orient="records", force_ascii=False)
# conversations_JSON_detect_results = json.loads(conversations_JSON_detect_results)

# 3. remove all the sentences that have length of conversation less than 5 words
# Function for detecting words

#[item["text"] for item in conversations_pharmacy_JSON if item.get("case_turn_id") == "1_4"]

# # with open("conversation_tested.json", "r", encoding="utf-8") as f:
# #     conversation_tested_JSON = json.load(f)
# with open("error_inserted_results_total.json", "r", encoding="utf-8") as f:
#     error_inserted_results_total_JSON = json.load(f)

# # parse it again to solve the issues of \\Chinese\\
# if isinstance(error_inserted_results_total_JSON, str):
#     error_inserted_results_total_JSON = json.loads(error_inserted_results_total_JSON)
# else:
#     error_inserted_results_total_JSON = error_inserted_results_total_JSON

# user_input = (
#     f"Original English Text: {conversation_tested_JSON[0]['text']}",
#     f"Original Chinese Translation: {conversation_tested_JSON[0]['text_cn']}")
# user_input_string = ','.join(user_input) # only use this!!! user_input -> gives tuple and it won't fit in the call_llm function

# Using dspy

In [ ]:
# 1. Define the language model
lm=dspy.LM(
    model="openrouter/openai/gpt-5-mini",
    api_key=api_key,
    temperature = 1,
    max_tokens=16000
)

In [ ]:
# 2. Register globally
dspy.settings.configure(lm = lm,track_usage=True) # track usage also help us to see which model is currently using
dspy.configure(lm=lm, track_usage=True)

# Error insertion

## Prepare dataset

In [ ]:
# 1. load the data
# folder 1. original data
from google.colab import files
uploaded = files.upload() # will open a window then you can select two files by using Ctrl conversation_tested.json

In [ ]:
# 2. load original conversation
original_conversations= pd.read_excel("original_conversations.xlsx")
original_conversations_JSON = original_conversations.to_json(orient="records", force_ascii=False)
original_conversations_JSON = json.loads(original_conversations_JSON)

In [ ]:
# 3. get the dataset with only the long conversation (words > 5 )
original_conversations_JSON_long = [
    item for item in original_conversations_JSON
    if isinstance(item.get("text"), str) and word_count(item.get("text")) >5
]
print(" rows before removal:", len(original_conversations_JSON),
      "\n rows after removal:", len(original_conversations_JSON_long))

## Inserting errors
- pharmacist -> patients (inserting errors to the Chinese text)
- patients -> pharmacist (inserting errors to the English text)

In [ ]:
# Define a DSPy signature - for Chinese error insertion (direction of conversation pharmacist -> patients) -
class InsertError(dspy.Signature):
# a. Provide the instruction
    """You are a medical translation error simulator.
Your task:
1. Select exactly THREE error types (error 1 error 2 and error 3) from the provided Error Table that logically fits the context of the given sentence.
2. Ensure the selected errors are **high quality**, based on the criteria below.
3. Insert error 1 into the orginal Chinese translation.
4. Insert error 2 into the orginal Chinese translation.
5. Insert error 3 into the orginal Chinese translation.
6. Update the English back-translation for these three sentences so that it clearly reflects the inserted error.
7. What is the best error insertion method?

## High Quality Errors
An error is considered high quality if it is:
- **Plausible**: Could realistically occur in medical translation (not random or nonsensical).
- **Clinically Impactful**: Affects diagnosis, treatment, dosage, safety, or patient understanding.
- **Subtle**: Not immediately obvious; requires careful checking to detect.

### **Error Table**
1. **MQM Core-Derived Errors**
- Mistranslation: Meaning deviates from the source.
- Overtranslation: Extra information added.
- Undertranslation: Content omitted.
- Addition: New content added without basis.
- Omission: Missing source content.
- Untranslated: Left in original language.
- Do Not Translate: Terms marked to remain untranslated were incorrectly translated.
- Terminology Error: Inaccurate or inconsistent medical term usage.

2. **Errors in Medical Conversations**
- Units & Dosage: Mistakes in drug quantity, units, or frequency.
- Temporal: Errors in timing or duration of treatment.
- Negation: Incorrect handling of negations (e.g., “not”).
- False Friends: Misleading similar-looking/sounding words across languages.
- Phonetic Confusion: Errors from misheard similar-sounding words.

3. **English ↔ Chinese Specific Errors**
- Cultural/Contextual: Misinterpretation of culturally specific phrases (e.g., traditional Chinese medicine).
- Idioms/Figurative: Literal translation of idiomatic expressions.
- Syntax/Grammar: Structural mismatches between Chinese and English.

4. **Identity checking errors**
- Patients name errors: “full name” replaced with “first name”, surname-first vs. given-name-first swapped, omitted or altered, nickname used instead of legal name, or variant romanisations (e.g. pinyin vs Wade-Giles).
- Date of birth errors: “date of birth” replaced with “age”, day or month order reversed, minor digit changes in year or month or day.
- MRN/ID errors: MRN/UR mistaken for Medicare/IHI/other ID, digits modified or dropped, MRN omitted when requested.
- General number errors: numbers misheard or reordered, digits added or removed, small but clinically important numeric changes.

"""


# b. Define the input
    en_1 = dspy.InputField(desc="Original English text of the conversation turn")
    cn_1 = dspy.InputField(desc="Original accurate Chinese translation")

# c. Define the output (as schema)
    errors_1 = dspy.OutputField(desc="Error category for error 1(e.g., Units & Dosage, Negation, Terminology, etc.)")
    errors_domain_1 = dspy.OutputField(desc="Which domain the error 1 belongs to")
    errors_explain_1 = dspy.OutputField(desc="Explanation of what was changed and why it is an error 1, in English")
    en_2_1 = dspy.OutputField(desc="English back-translation after the error 1 was introduced")
    cn_2_1 = dspy.OutputField(desc="Chinese translation with the error 1 inserted")

    errors_2 = dspy.OutputField(desc="Error category for error 2(e.g., Units & Dosage, Negation, Terminology, etc.)")
    errors_domain_2 = dspy.OutputField(desc="Which domain the error 2 belongs to")
    errors_explain_2 = dspy.OutputField(desc="Explanation of what was changed and why it is an error 2, in English")
    en_2_2 = dspy.OutputField(desc="English back-translation after the error 2 was introduced")
    cn_2_2 = dspy.OutputField(desc="Chinese translation with the error 2 inserted")

    errors_3 = dspy.OutputField(desc="Error category for error 3(e.g., Units & Dosage, Negation, Terminology, etc.)")
    errors_domain_3 = dspy.OutputField(desc="Which domain the error 3 belongs to")
    errors_explain_3 = dspy.OutputField(desc="Explanation of what was changed and why it is an error 3, in English")
    en_2_3 = dspy.OutputField(desc="English back-translation after the error 3 was introduced")
    cn_2_3 = dspy.OutputField(desc="Chinese translation with the error 3 inserted")

    error_best = dspy.OutputField(desc="The best error insertion method")
    error_best_explain = dspy.OutputField(desc="Explanation of the best error insertion method")

    ambiguity = dspy.OutputField(desc="Was any part of the instructions unclear? If so, please indicate what is unclear, so the author can add clarification.")


In [ ]:
# Create a model called insert_error
insert_error = dspy.Predict(InsertError) #normal prediction
#insert_error = dspy.ChainOfThought(InsertError) # using chain of thoughts

# can test the function here
# error_details = insert_error(en_1= conversation_tested_JSON[0]['text'], cn_1 = conversation_tested_JSON[0]['text_cn'])
# error_details
#   # f"Original English Text: {conversation_tested_JSON[0]['text']}",
#   #   f"Original Chinese Translation: {conversation_tested_JSON[0]['text_cn']}")

In [ ]:
# Define a DSPy signature - for English error insertion (direction of conversation patients -> pharmacist)
# Define a DSPy signature
class InsertError_patients(dspy.Signature):
# a. Provide the instruction
    """You are a medical translation error simulator.
Your task:
1. Select exactly THREE error types (error 1 error 2 and error 3) from the provided Error Table that logically fits the context of the given sentence.
2. Ensure the selected errors are **high quality**, based on the criteria below.
3. Insert error 1 into the orginal English translation (en_1).
4. Insert error 2 into the orginal English translation (en_1).
5. Insert error 3 into the orginal English translation (en_1).
6. Update the Chinese back-translation for these three sentences so that it clearly reflects the inserted error.
7. What is the best error insertion method?

## High Quality Errors
An error is considered high quality if it is:
- **Plausible**: Could realistically occur in medical translation (not random or nonsensical).
- **Clinically Impactful**: Affects diagnosis, treatment, dosage, safety, or patient understanding.
- **Subtle**: Not immediately obvious; requires careful checking to detect.

### **Error Table**
1. **MQM Core-Derived Errors**
- Mistranslation: Meaning deviates from the source.
- Overtranslation: Extra information added.
- Undertranslation: Content omitted.
- Addition: New content added without basis.
- Omission: Missing source content.
- Untranslated: Left in original language.
- Do Not Translate: Terms marked to remain untranslated were incorrectly translated.
- Terminology Error: Inaccurate or inconsistent medical term usage.

2. **Errors in Medical Conversations**
- Units & Dosage: Mistakes in drug quantity, units, or frequency.
- Temporal: Errors in timing or duration of treatment.
- Negation: Incorrect handling of negations (e.g., “not”).
- False Friends: Misleading similar-looking/sounding words across languages.
- Phonetic Confusion: Errors from misheard similar-sounding words.

3. **English ↔ Chinese Specific Errors**
- Cultural/Contextual: Misinterpretation of culturally specific phrases (e.g., traditional Chinese medicine).
- Idioms/Figurative: Literal translation of idiomatic expressions.
- Syntax/Grammar: Structural mismatches between Chinese and English.

4. **Identity checking errors**
- Patients name errors: “full name” replaced with “first name”, surname-first vs. given-name-first swapped, omitted or altered, nickname used instead of legal name, or variant romanisations (e.g. pinyin vs Wade-Giles).
- Date of birth errors: “date of birth” replaced with “age”, day or month order reversed, minor digit changes in year or month or day.
- MRN/ID errors: MRN/UR mistaken for Medicare/IHI/other ID, digits modified or dropped, MRN omitted when requested.
- General number errors: numbers misheard or reordered, digits added or removed, small but clinically important numeric changes.
"""

# b. Define the input
    en_1 = dspy.InputField(desc="Original English text of the conversation turn")
    cn_1 = dspy.InputField(desc="Original accurate Chinese translation")

# c. Define the output (as schema)
    errors_1 = dspy.OutputField(desc="Error category for error 1(e.g., Units & Dosage, Negation, Terminology, etc.)")
    errors_domain_1 = dspy.OutputField(desc="Which domain the error 1 belongs to")
    errors_explain_1 = dspy.OutputField(desc="Explanation of what was changed and why it is an error 1, in English")
    en_2_1 = dspy.OutputField(desc="English translation after the error 1 was introduced")
    cn_2_1 = dspy.OutputField(desc="Chinese back-translation with the error 1 inserted")

    errors_2 = dspy.OutputField(desc="Error category for error 2(e.g., Units & Dosage, Negation, Terminology, etc.)")
    errors_domain_2 = dspy.OutputField(desc="Which domain the error 2 belongs to")
    errors_explain_2 = dspy.OutputField(desc="Explanation of what was changed and why it is an error 2, in English")
    en_2_2 = dspy.OutputField(desc="English translation after the error 2 was introduced")
    cn_2_2 = dspy.OutputField(desc="Chinese back-translation with the error 2 inserted")

    errors_3 = dspy.OutputField(desc="Error category for error 3(e.g., Units & Dosage, Negation, Terminology, etc.)")
    errors_domain_3 = dspy.OutputField(desc="Which domain the error 3 belongs to")
    errors_explain_3 = dspy.OutputField(desc="Explanation of what was changed and why it is an error 3, in English")
    en_2_3 = dspy.OutputField(desc="English translation after the error 3 was introduced")
    cn_2_3 = dspy.OutputField(desc="Chinese back-translation with the error 3 inserted")

    error_best = dspy.OutputField(desc="The best error insertion method")
    error_best_explain = dspy.OutputField(desc="Explanation of the best error insertion method")

    ambiguity = dspy.OutputField(desc="Was any part of the instructions unclear? If so, please indicate what is unclear, so the author can add clarification.")


In [ ]:
insert_error_patients = dspy.Predict(InsertError_patients) #normal prediction

### For Pharmacist

In [ ]:
# get the data
conversations_pharmacy_JSON_Pharmacist = [item for item in original_conversations_JSON_long if item.get("speaker") == "Pharmacist"]
#conversations_pharmacy_JSON_Pharmacist # it is a list but still can be used efficiently
#conversation_tested_JSON_Oncologist[0]['text']

In [ ]:
Pharmacist_results = []

for turn_cal, item in enumerate(conversations_pharmacy_JSON_Pharmacist, start=1):
    error_details = insert_error(en_1=item['text'], cn_1=item['text_cn'])

    error_dict = {
        "case_turn_id": item['case_turn_id'],
        "setting_short": item['setting_short'],
        "speaker": item['speaker'],
        "en_1": item['text'],
        "cn_1": item['text_cn'],

        "errors_1": error_details.errors_1,
        "errors_domain_1": error_details.errors_domain_1,
        "errors_explain_1": error_details.errors_explain_1,
        "en_2_1": error_details.en_2_1,
        "cn_2_1": error_details.cn_2_1,

        "errors_2": error_details.errors_2,
        "errors_domain_2": error_details.errors_domain_2,
        "errors_explain_2": error_details.errors_explain_2,
        "en_2_2": error_details.en_2_2,
        "cn_2_2": error_details.cn_2_2,

        "errors_3": error_details.errors_3,
        "errors_domain_3": error_details.errors_domain_3,
        "errors_explain_3": error_details.errors_explain_3,
        "en_2_3": error_details.en_2_3,
        "cn_2_3": error_details.cn_2_3,

        "error_best": error_details.error_best,
        "error_best_explain": error_details.error_best_explain,

        "ambiguity": error_details.ambiguity
    }
    Pharmacist_results.append(error_dict)

print_results(error_details, turn_cal, Pharmacist_results)
Pharmacist_results_JSON = json.dumps(Pharmacist_results, ensure_ascii=False)
Pharmacist_results_JSON = json.loads(Pharmacist_results_JSON)
Pharmacist_results_JSON


current model: openrouter/openai/gpt-5-mini 
loop times: 33 
total generated results (rows): 33


[{'case_turn_id': '1_1',
  'setting_short': 'Discharge',
  'speaker': 'Pharmacist',
  'en_1': 'Hello, my name’s Corey, I’m one of the pharmacists here. Can I confirm your full name and DOB? ',
  'cn_1': '您好，我叫 Corey，是这里的药剂师之一。请问您的全名和出生日期是？',
  'errors_1': 'Patients name errors (first name vs full name)',
  'errors_domain_1': 'Identity checking errors',
  'errors_explain_1': 'The source asks for the patient\'s "full name." I changed the Chinese "全名" to the less specific "名字" (name/first name). This is a plausible, subtle identity-checking error: it reduces specificity (may omit surname), could lead to mismatched records, and is clinically important when verifying identity.',
  'en_2_1': 'Hello, my name’s Corey, I’m one of the pharmacists here. Can I confirm your name and DOB?',
  'cn_2_1': '您好，我叫 Corey，是这里的药剂师之一。请问您的名字和出生日期是？',
  'errors_2': 'Date of birth error (DOB replaced by age)',
  'errors_domain_2': 'Identity checking errors',
  'errors_explain_2': 'I replaced "出生日期" (date of bir

In [ ]:
# summary of errors
counter_summary_clean(Pharmacist_results_JSON, ["errors_1", "errors_2", "errors_3"], True)

summary of records: 99 
summary of cleaned records: 99


[('Negation', 24),
 ('Terminology Error', 22),
 ('Units & Dosage', 12),
 ('Temporal', 8),
 ('Omission', 8),
 ('Addition', 6),
 ('Temporal Error', 3),
 ('Mistranslation', 3),
 ('Patients name errors', 2),
 ('Date of birth error', 2),
 ('False Friend / Mistranslation', 1),
 ('Date of birth errors', 1),
 ('MRN/ID errors', 1),
 ('False Friends / Terminology Error', 1),
 ('General number error', 1),
 ('Terminology / Mistranslation', 1),
 ("Patient's name error", 1),
 ('Units & Dosage / General number error', 1),
 ('General Number Error', 1)]

### For Patients

In [ ]:
# get the data, they will use different class
conversations_pharmacy_JSON_Patients = [item for item in original_conversations_JSON_long if item.get("speaker") == "Patient" or item.get("speaker") == "Parent"]

In [ ]:
Patients_results = []

for turn_cal, item in enumerate(conversations_pharmacy_JSON_Patients, start=1):
    error_details = insert_error_patients(en_1=item['text'], cn_1=item['text_cn'])

    error_dict = {
        "case_turn_id": item['case_turn_id'],
        "setting_short": item['setting_short'],
        "speaker": item['speaker'],
        "en_1": item['text'],
        "cn_1": item['text_cn'],

        "errors_1": error_details.errors_1,
        "errors_domain_1": error_details.errors_domain_1,
        "errors_explain_1": error_details.errors_explain_1,
        "en_2_1": error_details.en_2_1,
        "cn_2_1": error_details.cn_2_1,

        "errors_2": error_details.errors_2,
        "errors_domain_2": error_details.errors_domain_2,
        "errors_explain_2": error_details.errors_explain_2,
        "en_2_2": error_details.en_2_2,
        "cn_2_2": error_details.cn_2_2,

        "errors_3": error_details.errors_3,
        "errors_domain_3": error_details.errors_domain_3,
        "errors_explain_3": error_details.errors_explain_3,
        "en_2_3": error_details.en_2_3,
        "cn_2_3": error_details.cn_2_3,

        "error_best": error_details.error_best,
        "error_best_explain": error_details.error_best_explain,

        "ambiguity": error_details.ambiguity
    }

    Patients_results.append(error_dict)


Patients_results_JSON = json.dumps(Patients_results, ensure_ascii=False)
Patients_results_JSON = json.loads(Patients_results_JSON)
Patients_results_JSON


[{'case_turn_id': '1_2',
  'setting_short': 'Discharge',
  'speaker': 'Patient',
  'en_1': 'My name is Kaiyu, and my DOB is 29/04/1997',
  'cn_1': '我的名字叫kaiyu，我的出生日期是1997年4月29日。',
  'errors_1': 'Patients name error (nickname/omission of part of name)',
  'errors_domain_1': 'Identity checking errors — Patients name errors',
  'errors_explain_1': 'The original given name "Kaiyu" was shortened to "Kai", omitting the final syllable. This is a plausible spoken-translation error (a syllable dropped or a nickname used). It changes the patient\'s legal identifier and can lead to misidentification in records, appointments, or medication administration — clinically important and subtle enough to be missed.',
  'en_2_1': 'My name is Kai, and my DOB is 29/04/1997',
  'cn_2_1': '我的名字叫Kai，我的出生日期是1997年4月29日。',
  'errors_2': 'Date of birth digit error (year altered)',
  'errors_domain_2': 'Identity checking errors — Date of birth errors / General number errors',
  'errors_explain_2': "The year in the 

In [ ]:
print_results(error_details, turn_cal, Patients_results)

current model: openrouter/openai/gpt-5-mini 
loop times: 16 
total generated results (rows): 16


In [ ]:
counter_summary_clean(Patients_results_JSON, ["errors_1", "errors_2", "errors_3"], True)


summary of records: 48 
summary of cleaned records: 48


[('Negation', 12),
 ('Addition', 8),
 ('Temporal', 7),
 ('Units & Dosage', 5),
 ('Terminology Error', 4),
 ('General number errors', 2),
 ('Patients name error', 1),
 ('Date of birth digit error', 1),
 ('Date of birth replaced with age', 1),
 ('Undertranslation / Omission', 1),
 ('Date of birth error', 1),
 ('Patient name error — addition', 1),
 ('General number error', 1),
 ('Mistranslation', 1),
 ('Overtranslation', 1),
 ('Negation error', 1)]

### Combine list and save

In [ ]:
# combine two lists
original_conversations_errors = Pharmacist_results_JSON + Patients_results_JSON

In [ ]:
original_conversations_errors = pd.DataFrame(original_conversations_errors)
original_conversations_errors.to_excel("original_conversations_errors.xlsx", index=False)
files.download("original_conversations_errors.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Error detection

## Prepare dataset

In [ ]:
from google.colab import files
# folder 2. error_inserted
uploaded = files.upload() # will open a window then you can select two files by using Ctrl conversation_tested.json

Saving original_conversations_errors.xlsx to original_conversations_errors.xlsx


In [ ]:
original_conversations_errors= pd.read_excel("original_conversations_errors.xlsx")
original_conversations_errors_JSON = original_conversations_errors.to_json(orient="records", force_ascii=False)
original_conversations_errors_JSON = json.loads(original_conversations_errors_JSON)

## DSPy signature

In [ ]:
# Define a DSPy signature
class DetectError(dspy.Signature):
# a. Provide the instruction
    """You are an expert in medical translation evaluation.

You will be given:
1. Two original Text: a sentence from a doctor–patient conversation.
2. Translation: the translated version.
3. Error Table: a list of possible translation error types.
4. Criteria for Serious Clinical Consequences.

Your task:
- Compare the Original and Translation.
- Detect any translation error based on the Error Table.
- Identify the domain of the error (e.g., MQM Core, Medical Conversation, or English ↔ Chinese Specific).
- Explain the reason why it is an error.
- Determine whether the error is associated with a Serious Clinical Consequence (Y/N).
- Explain what kind of clinical consequence it is, according to the Criteria for Serious Clinical Consequences

### **Error Table**
1. **MQM Core-Derived Errors**
- Mistranslation: Meaning deviates from the source.
- Overtranslation: Extra information added.
- Undertranslation: Content omitted.
- Addition: New content added without basis.
- Omission: Missing source content.
- Untranslated: Left in original language.
- Do Not Translate: Terms marked to remain untranslated were incorrectly translated.
- Terminology Error: Inaccurate or inconsistent medical term usage.

2. **Errors in Medical Conversations**
- Units & Dosage & Temporal: Mistakes in drug quantity, units, or frequency, or errors in timing or duration of treatment.
- Negation: Incorrect handling of negations (e.g., “not”).
- False Friends: Misleading similar-looking/sounding words across languages.
- Phonetic Confusion: Errors from misheard similar-sounding words.

3. **English ↔ Chinese Specific Errors**
- Cultural/Contextual: Misinterpretation of culturally specific phrases (e.g., traditional Chinese medicine).
- Idioms/Figurative: Literal translation of idiomatic expressions.
- Syntax/Grammar: Structural mismatches between Chinese and English.

4. **Identity checking errors**
- Patients name errors: “full name” replaced with “first name”, surname-first vs. given-name-first swapped, omitted or altered, nickname used instead of legal name, or variant romanisations (e.g. pinyin vs Wade-Giles).
- Date of birth errors: “date of birth” replaced with “age”, day or month order reversed, minor digit changes in year or month or day.
- MRN/ID errors: MRN/UR mistaken for Medicare/IHI/other ID, digits modified or dropped, MRN omitted when requested.
- General number errors: numbers misheard or reordered, digits added or removed, small but clinically important numeric changes.
---

### **Criteria for Serious Clinical Consequences**

An error is considered clinically significant if it affects:
1. The **history of present illness**
2. The **past medical history**
3. Any **diagnostic or therapeutic intervention**
4. **Plans for follow-up or future care**

"""

# b. Define the input
    #right english, wrong chinese
    en_1 = dspy.InputField(desc="Original text 1")
    cn_2_1 = dspy.InputField(desc="Translated text 1") # only try this one first
    #right chinese, wrong english
    cn_1 = dspy.InputField(desc="Original text 2")
    en_2_1 = dspy.InputField(desc="Translated text 2") # only try this one first


    # cn_1 = dspy.InputField(desc="Original Chinese")
    # en_2_1 = dspy.InputField(desc="Translated English")


# c. Define the output (as schema)
    errors_detected_text1 = dspy.OutputField(desc="The specific type of translation error identified from text 1/translated text 1, such as 'Units & Dosage', 'Negation', 'Terminology Error', etc.")
    errors_detected_domain_text1 = dspy.OutputField(desc="The category or domain the error falls under from text 1/translated text 1, such as 'MQM Core-Derived Errors', 'Errors in Medical Conversations', or 'English ↔ Chinese Specific Errors'.")
    errors_detected_domain_explain_text1  = dspy.OutputField(desc="Explanation why do you think it is an error.")
    clinical_consequences_explain_text1 = dspy.OutputField(desc="A brief explanation of how the error could lead to a clinical consequence.")
    clinical_consequences_flag_text1 = dspy.OutputField(desc="Flag indicating whether the error may lead to serious clinical consequences (Y = Yes, N = No).")

    errors_detected_text2 = dspy.OutputField(desc="The specific type of translation error identified from text 2/translated text 2, such as 'Units & Dosage', 'Negation', 'Terminology Error', etc.")
    errors_detected_domain_text2 = dspy.OutputField(desc="The category or domain the error falls under from text 2/translated text 2, such as 'MQM Core-Derived Errors', 'Errors in Medical Conversations', or 'English ↔ Chinese Specific Errors'.")
    errors_detected_domain_explain_text2  = dspy.OutputField(desc="Explanation why do you think it is an error.")
    clinical_consequences_explain_text2 = dspy.OutputField(desc="A brief explanation of how the error could lead to a clinical consequence.")
    clinical_consequences_flag_text2 = dspy.OutputField(desc="Flag indicating whether the error may lead to serious clinical consequences (Y = Yes, N = No).")

    ambiguity = dspy.OutputField(desc="Was any part of the instructions unclear? If so, please indicate what is unclear, so the author can add clarification.")


In [ ]:
# # Create a model called insert_error
# detect_error = dspy.Predict(DetectError) #normal prediction

## Configuration of different models

In [ ]:
# gemini
lm.gm=dspy.LM(
    model="openrouter/google/gemini-2.5-flash-lite",
    api_key=api_key,
    temperature = 1,
    max_tokens=16000
)

In [ ]:
# Qwen
lm.qwen=dspy.LM(
    model="openrouter/qwen/qwen3-235b-a22b-2507",
    api_key=api_key,
    temperature = 1,
    max_tokens=16000
)

In [ ]:
# Mistral
lm.mistral=dspy.LM(
    model="openrouter/mistralai/devstral-2512:free",
    api_key=api_key,
    temperature = 1,
    max_tokens=16000
)

In [ ]:
# deepseek
lm.deepseek=dspy.LM(
    model="openrouter/deepseek/deepseek-v3.2",
    api_key=api_key,
    temperature = 1,
    max_tokens=16000
)

## Call functions

In [ ]:
# predictio tasks row by one row
# usage: prediction_tasks(DetectError, original_conversations_errors_JSON[0])
def prediction_tasks(class_Signature, input_json):
  prediction = dspy.Predict(class_Signature)

  result_json = prediction(
      en_1=input_json['en_1'],
      cn_2_1 =input_json['cn_2_1'],
      cn_1=input_json['cn_1'],
      en_2_1 =input_json['en_2_1']
  )
  return result_json


In [ ]:
def loop_llm(class_Signature, data_input, new_model=None):
    final_data = []

    for turn_cal, item in enumerate(data_input, start=1):
        # Use context if both conditions are met
        if  new_model is not None:
            with dspy.context(lm=new_model):
              result_row = prediction_tasks(class_Signature, item)
              context_status = "yes"
              current_model = dspy.settings.lm.history[0]['model']
        else:
            result_row = prediction_tasks(class_Signature, item)
            context_status = "no"
            current_model = dspy.settings.lm.history[0]['model']

        # Get completion data
        result_row_completion = result_row._completions[0]._store

        # Merge with item
        result_row_dict = item | result_row_completion

        final_data.append(result_row_dict)

        # Print progress
        print("current model:", current_model,
              "\n",
              "loops:", turn_cal,
              "\n",
              "total generated results (rows):", len(final_data),
              "\n",
              "context:", context_status)

    return final_data

In [ ]:
detect_error_dict = [
    {key: data[key] for key in ["case_turn_id", "setting_short","speaker", "en_1", "cn_2_1", "cn_1", "en_2_1", "errors_1", "errors_domain_1", "errors_explain_1"]}
    for data in original_conversations_errors_JSON
]


## Results

In [ ]:
test_gm = loop_llm(DetectError,detect_error_dict, new_model=lm.gm)

current model: openrouter/google/gemini-2.5-flash-lite 
 loops: 1 
 total generated results (rows): 1 
 context: yes
current model: openrouter/google/gemini-2.5-flash-lite 
 loops: 2 
 total generated results (rows): 2 
 context: yes
current model: openrouter/google/gemini-2.5-flash-lite 
 loops: 3 
 total generated results (rows): 3 
 context: yes
current model: openrouter/google/gemini-2.5-flash-lite 
 loops: 4 
 total generated results (rows): 4 
 context: yes
current model: openrouter/google/gemini-2.5-flash-lite 
 loops: 5 
 total generated results (rows): 5 
 context: yes
current model: openrouter/google/gemini-2.5-flash-lite 
 loops: 6 
 total generated results (rows): 6 
 context: yes
current model: openrouter/google/gemini-2.5-flash-lite 
 loops: 7 
 total generated results (rows): 7 
 context: yes
current model: openrouter/google/gemini-2.5-flash-lite 
 loops: 8 
 total generated results (rows): 8 
 context: yes
current model: openrouter/google/gemini-2.5-flash-lite 
 loops: 

In [ ]:
test_qwen = loop_llm(DetectError,detect_error_dict, new_model=lm.qwen)

current model: openrouter/qwen/qwen3-235b-a22b-2507 
 loops: 1 
 total generated results (rows): 1 
 context: yes
current model: openrouter/qwen/qwen3-235b-a22b-2507 
 loops: 2 
 total generated results (rows): 2 
 context: yes
current model: openrouter/qwen/qwen3-235b-a22b-2507 
 loops: 3 
 total generated results (rows): 3 
 context: yes
current model: openrouter/qwen/qwen3-235b-a22b-2507 
 loops: 4 
 total generated results (rows): 4 
 context: yes
current model: openrouter/qwen/qwen3-235b-a22b-2507 
 loops: 5 
 total generated results (rows): 5 
 context: yes
current model: openrouter/qwen/qwen3-235b-a22b-2507 
 loops: 6 
 total generated results (rows): 6 
 context: yes
current model: openrouter/qwen/qwen3-235b-a22b-2507 
 loops: 7 
 total generated results (rows): 7 
 context: yes
current model: openrouter/qwen/qwen3-235b-a22b-2507 
 loops: 8 
 total generated results (rows): 8 
 context: yes
current model: openrouter/qwen/qwen3-235b-a22b-2507 
 loops: 9 
 total generated results

In [ ]:
test_mistral = loop_llm(DetectError,detect_error_dict, new_model=lm.mistral)

current model: openrouter/mistralai/devstral-2512:free 
 loops: 1 
 total generated results (rows): 1 
 context: yes
current model: openrouter/mistralai/devstral-2512:free 
 loops: 2 
 total generated results (rows): 2 
 context: yes
current model: openrouter/mistralai/devstral-2512:free 
 loops: 3 
 total generated results (rows): 3 
 context: yes
current model: openrouter/mistralai/devstral-2512:free 
 loops: 4 
 total generated results (rows): 4 
 context: yes
current model: openrouter/mistralai/devstral-2512:free 
 loops: 5 
 total generated results (rows): 5 
 context: yes
current model: openrouter/mistralai/devstral-2512:free 
 loops: 6 
 total generated results (rows): 6 
 context: yes
current model: openrouter/mistralai/devstral-2512:free 
 loops: 7 
 total generated results (rows): 7 
 context: yes
current model: openrouter/mistralai/devstral-2512:free 
 loops: 8 
 total generated results (rows): 8 
 context: yes
current model: openrouter/mistralai/devstral-2512:free 
 loops: 

In [ ]:
test_default = loop_llm(DetectError,detect_error_dict, new_model=None)

current model: openrouter/openai/gpt-5-mini 
 loops: 1 
 total generated results (rows): 1 
 context: no
current model: openrouter/openai/gpt-5-mini 
 loops: 2 
 total generated results (rows): 2 
 context: no
current model: openrouter/openai/gpt-5-mini 
 loops: 3 
 total generated results (rows): 3 
 context: no
current model: openrouter/openai/gpt-5-mini 
 loops: 4 
 total generated results (rows): 4 
 context: no
current model: openrouter/openai/gpt-5-mini 
 loops: 5 
 total generated results (rows): 5 
 context: no
current model: openrouter/openai/gpt-5-mini 
 loops: 6 
 total generated results (rows): 6 
 context: no
current model: openrouter/openai/gpt-5-mini 
 loops: 7 
 total generated results (rows): 7 
 context: no
current model: openrouter/openai/gpt-5-mini 
 loops: 8 
 total generated results (rows): 8 
 context: no
current model: openrouter/openai/gpt-5-mini 
 loops: 9 
 total generated results (rows): 9 
 context: no
current model: openrouter/openai/gpt-5-mini 
 loops: 10

In [ ]:
test_deepseek= loop_llm(DetectError,detect_error_dict, new_model=lm.deepseek)

current model: openrouter/deepseek/deepseek-v3.2 
 loops: 1 
 total generated results (rows): 1 
 context: yes
current model: openrouter/deepseek/deepseek-v3.2 
 loops: 2 
 total generated results (rows): 2 
 context: yes
current model: openrouter/deepseek/deepseek-v3.2 
 loops: 3 
 total generated results (rows): 3 
 context: yes
current model: openrouter/deepseek/deepseek-v3.2 
 loops: 4 
 total generated results (rows): 4 
 context: yes
current model: openrouter/deepseek/deepseek-v3.2 
 loops: 5 
 total generated results (rows): 5 
 context: yes
current model: openrouter/deepseek/deepseek-v3.2 
 loops: 6 
 total generated results (rows): 6 
 context: yes
current model: openrouter/deepseek/deepseek-v3.2 
 loops: 7 
 total generated results (rows): 7 
 context: yes
current model: openrouter/deepseek/deepseek-v3.2 
 loops: 8 
 total generated results (rows): 8 
 context: yes
current model: openrouter/deepseek/deepseek-v3.2 
 loops: 9 
 total generated results (rows): 9 
 context: yes
c

In [ ]:
test_default_pd = pd.DataFrame(test_default)
test_qwen_pd = pd.DataFrame(test_qwen)
test_gm_pd = pd.DataFrame(test_gm)
test_mistral_pd = pd.DataFrame(test_mistral)
test_deepseek_pd = pd.DataFrame(test_deepseek)
original_conversation= pd.DataFrame(detect_error_dict)

In [ ]:
original_conversation

,case_turn_id,setting_short,speaker,en_1,cn_2_1,cn_1,en_2_1,errors_1,errors_domain_1,errors_explain_1
0,1_1,Discharge,Pharmacist,"Hello, my name’s Corey, I’m one of the pharmac...",您好，我叫 Corey，是这里的药剂师之一。请问您的名字和出生日期是？,您好，我叫 Corey，是这里的药剂师之一。请问您的全名和出生日期是？,"Hello, my name’s Corey, I’m one of the pharmac...",Patients name errors (first name vs full name),Identity checking errors,"The source asks for the patient's ""full name.""..."
1,1_3,Discharge,Pharmacist,Thank you Kaiyu. And is now a good time to run...,谢谢你，Kaiyu。现在方便在你回家后一起复查一下你目前的所有药物吗?,谢谢你，Kaiyu。现在方便在你回家前一起复查一下你目前的所有药物吗?,Thank you Kaiyu. And is now a good time to run...,Temporal,Medical Conversations — Temporal,"The phrase ""回家前"" (before you go home) was chan..."
2,1_5,Discharge,Pharmacist,"Great. Before we start, can I just check, do y...",很好。在我们开始之前，我想先确认一下：你对药物没有过敏，或者以前用药时没有不良反应吗？,很好。在我们开始之前，我想先确认一下：你有没有对药物过敏，或者以前用药时有过不良反应？,"Great. Before we start, I just want to confirm...",Negation,Errors in Medical Conversations,The original question asks whether the patient...
3,1_7,Discharge,Pharmacist,Okay. The doctor has prescribed you an antibio...,好的。医生给你开了一种名为阿莫西林的抗生素，常见的商品名是 Augmentin Duo Fo...,好的。医生给你开了一种名为阿莫西林配克拉维酸的抗生素，常见的商品名是 Augmentin D...,Okay. The doctor has prescribed you an antibio...,Omission,MQM Core-Derived Errors,"I removed ""clavulanic acid"" (克拉维酸) from the dr..."
4,1_8,Discharge,Pharmacist,The dose is one tablet every morning and night...,剂量是早晚各一片，共服用 7 周。最好随餐服用，这样有助于吸收，也能减少胃部不适。比如，你可...,剂量是早晚各一片，共服用 7 天。最好随餐服用，这样有助于吸收，也能减少胃部不适。比如，你可...,The dose is one tablet every morning and night...,Units & Dosage,Errors in Medical Conversations,"The duration ""7 天"" (7 days) was changed to ""7 ..."
5,1_9,Discharge,Pharmacist,"Most people don’t have any major side-effects,...",大多数人会出现严重的副作用，但有些人可能会出现大便稀（或腹泻）、轻微的恶心或胃部不适。不过这...,大多数人不会出现严重的副作用，但有些人可能会出现大便稀（或腹泻）、轻微的恶心或胃部不适。不过...,"Most people will have major side-effects, but ...",Negation,Errors in Medical Conversations - Negation,"The negative particle ""不会"" (will not / don't) ..."
6,1_10,Discharge,Pharmacist,"If you notice any skin rash, swelling or diffi...",如果你发现自己起了皮疹、出现肿胀，或者有呼吸困难的情况，请继续服药，并马上去看医生或直接来医...,如果你发现自己起了皮疹、出现肿胀，或者有呼吸困难的情况，请立刻停药，并马上去看医生或直接来医...,"If you notice any skin rash, swelling or diffi...",Negation,Errors in Medical Conversations — Negation,"The instruction to ""stop the medicine"" (停药) ha..."
7,1_11,Discharge,Pharmacist,"It’s important you finish the full course, eve...",即使你感觉好转了，也不必把整个疗程的药吃完，这样才能把感染彻底治好,即使你感觉好转了，也一定要把整个疗程的药吃完，这样才能把感染彻底治好,"Even if you're feeling better, you don't have ...",Negation,Errors in Medical Conversations - Negation,"The original affirmative instruction (""be sure..."
8,1_12,Discharge,Pharmacist,Please see your doctor if: Your pain or sympto...,如果出现以下情况，请去看医生：你的疼痛或症状加重、出现严重腹泻（尤其是水样或带血的腹泻）、或...,如果出现以下情况，请去看医生：你的疼痛或症状加重、出现严重腹泻（尤其是水样或带血的腹泻）、或...,Please see your doctor if: Your pain or sympto...,Negation,Errors in Medical Conversations,"The clause ""有任何让你感到担心的副作用"" (""any side effects ..."
9,2_1,Dose tapering,Pharmacist,"Hi Grace, my name is Lee, I’m one of the pharm...",“嗨 Grace，我叫Lee，是这里的药剂师之一。我想花几分钟时间和你讲一下你的抗焦虑药物，...,“嗨 Grace，我叫Lee，是这里的药剂师之一。我想花几分钟时间和你讲一下你的抗抑郁药物，...,"Hi Grace, my name is Lee, I’m one of the pharm...",Terminology Error,MQM Core-Derived: Terminology,"The term ""抗抑郁药物"" (antidepressant) was changed ..."


In [ ]:
final_detection = pd.concat(
    [
        original_conversation.reset_index(drop=True),
        test_default_pd.iloc[:, 12:21].reset_index(drop=True).add_suffix("_gpt"),
        test_qwen_pd.iloc[:, 12:21].reset_index(drop=True).add_suffix("_qwen"),
        test_gm_pd.iloc[:, 12:21].reset_index(drop=True).add_suffix("_gm"),
        test_mistral_pd.iloc[:, 12:21].reset_index(drop=True).add_suffix("_mistral"),
        test_deepseek_pd.iloc[:, 12:21].reset_index(drop=True).add_suffix("_deepseek")
    ],
    axis=1
)

In [ ]:
final_detection

,case_turn_id,setting_short,speaker,en_1,cn_2_1,cn_1,en_2_1,errors_1,errors_domain_1,errors_explain_1,...,ambiguity_mistral,errors_detected_domain_explain_text1_deepseek,clinical_consequences_explain_text1_deepseek,clinical_consequences_flag_text1_deepseek,errors_detected_text2_deepseek,errors_detected_domain_text2_deepseek,errors_detected_domain_explain_text2_deepseek,clinical_consequences_explain_text2_deepseek,clinical_consequences_flag_text2_deepseek,ambiguity_deepseek
0,1_1,Discharge,Pharmacist,"Hello, my name’s Corey, I’m one of the pharmac...",您好，我叫 Corey，是这里的药剂师之一。请问您的名字和出生日期是？,您好，我叫 Corey，是这里的药剂师之一。请问您的全名和出生日期是？,"Hello, my name’s Corey, I’m one of the pharmac...",Patients name errors (first name vs full name),Identity checking errors,"The source asks for the patient's ""full name.""...",...,No ambiguity in the instructions.,"The source text ""Can I confirm your full name ...",This omission could lead to incorrect patient ...,Y,Omission,Identity checking errors,"The source text ""请问您的全名和出生日期是？"" (""Can I confir...","As with the first pair, the omission of 'full'...",Y,The instructions are clear. No ambiguity was d...
1,1_3,Discharge,Pharmacist,Thank you Kaiyu. And is now a good time to run...,谢谢你，Kaiyu。现在方便在你回家后一起复查一下你目前的所有药物吗?,谢谢你，Kaiyu。现在方便在你回家前一起复查一下你目前的所有药物吗?,Thank you Kaiyu. And is now a good time to run...,Temporal,Medical Conversations — Temporal,"The phrase ""回家前"" (before you go home) was chan...",...,No ambiguity detected. The instructions were c...,"The original English phrase ""before you go hom...",If a patient believes the medicine review is t...,Y,Mistranslation (Temporal Error),Errors in Medical Conversations,"The original Chinese phrase ""在你回家前"" (before yo...",This error has the same consequence as in Text...,Y,The instructions are clear.
2,1_5,Discharge,Pharmacist,"Great. Before we start, can I just check, do y...",很好。在我们开始之前，我想先确认一下：你对药物没有过敏，或者以前用药时没有不良反应吗？,很好。在我们开始之前，我想先确认一下：你有没有对药物过敏，或者以前用药时有过不良反应？,"Great. Before we start, I just want to confirm...",Negation,Errors in Medical Conversations,The original question asks whether the patient...,...,No ambiguity detected.,"The original English question ""do you have any...",A leading question about allergies can cause a...,Y,Negation,Errors in Medical Conversations,"The original Chinese question ""你有没有对药物过敏，或者以前用...","Similar to the error in Text 1, phrasing the a...",Y,The instructions are clear. No ambiguity.
3,1_7,Discharge,Pharmacist,Okay. The doctor has prescribed you an antibio...,好的。医生给你开了一种名为阿莫西林的抗生素，常见的商品名是 Augmentin Duo Fo...,好的。医生给你开了一种名为阿莫西林配克拉维酸的抗生素，常见的商品名是 Augmentin D...,Okay. The doctor has prescribed you an antibio...,Omission,MQM Core-Derived Errors,"I removed ""clavulanic acid"" (克拉维酸) from the dr...",...,No ambiguity detected.,"The original English text specifies ""an antibi...",Amoxicillin-clavulanic acid (a beta-lactamase ...,Y,Omission / Undertranslation,MQM Core-Derived Errors,"The original Chinese text specifies ""阿莫西林**配克拉...",This error is identical in clinical impact to ...,Y,The instructions are clear.
4,1_8,Discharge,Pharmacist,The dose is one tablet every morning and night...,剂量是早晚各一片，共服用 7 周。最好随餐服用，这样有助于吸收，也能减少胃部不适。比如，你可...,剂量是早晚各一片，共服用 7 天。最好随餐服用，这样有助于吸收，也能减少胃部不适。比如，你可...,The dose is one tablet every morning and night...,Units & Dosage,Errors in Medical Conversations,"The duration ""7 天"" (7 days) was changed to ""7 ...",...,No ambiguity detected.,"The original English text specifies ""for 7 day...",Prescribing medication for 7 weeks instead of ...,Y,Mistranslation,MQM Core-Derived Errors,"The original Chinese text specifies ""共服用 7 天"" ...",Translating a 7-day course as a 7-week course ...,Y,The instructions were clear. The error in both...
5,1_9,Discharge,Pharmacist,"Most people don’t have any major side-effects,...",大多数人会出现严重的副作用，但有些人可能会出现大便稀（或腹泻）、轻微的恶心或胃部不适。不过这...,大多数人不会出现严重的副作用，但有些人可能会出现大便稀（或腹泻）、轻微的恶心或胃部不适。不过...,"Most people will have major side-effects, but ...",Negation,Errors in Medical Conversations - Negation,"Th

In [ ]:
final_detection.to_excel("final_detection.xlsx", index=False)
files.download("final_detection.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# orginal model openrouter/openai/gpt-5-mini

In [ ]:
conversations_JSON_detect_results = []

for turn_cal, item in enumerate(original_conversations_errors_JSON[0:1], start=1):
    detect_error_details = detect_error(
        en_1=item['en_1'],
        cn_2_1 =item['cn_2_1'],
        cn_1=item['cn_1'],
        en_2_1 =item['en_2_1']
        # ,
        # cn_1=item['cn_1'],
        # en_2_1=item['en_2_1']
        )

    detect_error_dict = {
        "case_turn_id": item['case_turn_id'],
        "setting_short": item['setting_short'],
        "speaker": item['speaker'],
        "errors": item['errors_1'],
        "errors_domain": item['errors_domain_1'],
        "errors_explain": item['errors_explain_1'],

        "en_1": item['en_1'],
        "cn_2_1": item['cn_2_1'],
        "errors_detected_text1": detect_error_details.errors_detected_text1,
        "errors_detected_domain_text1": detect_error_details.errors_detected_domain_text1,
        "errors_detected_domain_explain_text1": detect_error_details.errors_detected_domain_explain_text1,
        "clinical_consequences_explain_text1": detect_error_details.clinical_consequences_explain_text1,
        "clinical_consequences_flag_text1": detect_error_details.clinical_consequences_flag_text1,

        "cn_1": item['cn_1'],
        "en_2_1": item['en_2_1'],
        "errors_detected_text2": detect_error_details.errors_detected_text2,
        "errors_detected_domain_text2": detect_error_details.errors_detected_domain_text2,
        "errors_detected_domain_explain_text2": detect_error_details.errors_detected_domain_explain_text2,
        "clinical_consequences_explain_text2": detect_error_details.clinical_consequences_explain_text2,
        "clinical_consequences_flag_text2": detect_error_details.clinical_consequences_flag_text2,

        "ambiguity": detect_error_details.ambiguity
    }

    conversations_JSON_detect_results.append(detect_error_dict)

# print_results(detect_error_details, turn_cal, conversations_JSON_detect_results)
#counter_summary_clean(Patients_results_JSON, ["errors_1", "errors_2", "errors_3"], True)
#print_results(detect_error_details, turn_cal, conversations_JSON_detect_results)

# print("current model:", dspy.settings.lm.model,
#       "\nloop times:", turn_cal,
#       "\ntotal generated results (rows):", len(conversations_JSON_detect_results))

In [ ]:
test = loop_llm(DetectError,detect_error_dict[0:1], new_model=None, dspycontext=None)

current model: openrouter/openai/gpt-5-mini 
 loops: 1 
 total generated results (rows): 1 
 context: no


In [ ]:
test = loop_llm(DetectError,detect_error_dict[0:1], new_model=lm.gm)

current model: openrouter/google/gemini-2.5-flash-lite 
 loops: 1 
 total generated results (rows): 1 
 context: yes


In [ ]:
test = loop_llm(DetectError,detect_error_dict[0:1], new_model=None)

current model: openrouter/openai/gpt-5-mini 
 loops: 1 
 total generated results (rows): 1 
 context: no


In [ ]:
detect_error_dict

[{'case_turn_id': '1_1',
  'setting_short': 'Discharge',
  'speaker': 'Pharmacist',
  'errors_1': 'Patients name errors (first name vs full name)',
  'errors_domain_1': 'Identity checking errors',
  'errors_explain_1': 'The source asks for the patient\'s "full name." I changed the Chinese "全名" to the less specific "名字" (name/first name). This is a plausible, subtle identity-checking error: it reduces specificity (may omit surname), could lead to mismatched records, and is clinically important when verifying identity.',
  'en_1': 'Hello, my name’s Corey, I’m one of the pharmacists here. Can I confirm your full name and DOB? ',
  'cn_2_1': '您好，我叫 Corey，是这里的药剂师之一。请问您的名字和出生日期是？'},
 {'case_turn_id': '1_3',
  'setting_short': 'Discharge',
  'speaker': 'Pharmacist',
  'errors_1': 'Temporal',
  'errors_domain_1': 'Medical Conversations — Temporal',
  'errors_explain_1': 'The phrase "回家前" (before you go home) was changed to "回家后" (after you go home). This reverses the timing of when the medicat

In [ ]:
conversations_JSON_detect_results = []
model_name = None

for turn_cal, item in enumerate(original_conversations_errors_JSON[0:1], start=1):

  with dspy.context(lm=lm.gm):
    detect_error_details = detect_error(
        en_1=item['en_1'],
        cn_2_1 =item['cn_2_1'],
        cn_1=item['cn_1'],
        en_2_1 =item['en_2_1']
        # ,
        # cn_1=item['cn_1'],
        # en_2_1=item['en_2_1']
        )

    detect_error_dict = {
        "case_turn_id": item['case_turn_id'],
        "setting_short": item['setting_short'],
        "speaker": item['speaker'],
        "errors": item['errors_1'],
        "errors_domain": item['errors_domain_1'],
        "errors_explain": item['errors_explain_1'],

        "en_1": item['en_1'],
        "cn_2_1": item['cn_2_1'],
        "errors_detected_text1": detect_error_details.errors_detected_text1,
        "errors_detected_domain_text1": detect_error_details.errors_detected_domain_text1,
        "errors_detected_domain_explain_text1": detect_error_details.errors_detected_domain_explain_text1,
        "clinical_consequences_explain_text1": detect_error_details.clinical_consequences_explain_text1,
        "clinical_consequences_flag_text1": detect_error_details.clinical_consequences_flag_text1,

        "cn_1": item['cn_1'],
        "en_2_1": item['en_2_1'],
        "errors_detected_text2": detect_error_details.errors_detected_text2,
        "errors_detected_domain_text2": detect_error_details.errors_detected_domain_text2,
        "errors_detected_domain_explain_text2": detect_error_details.errors_detected_domain_explain_text2,
        "clinical_consequences_explain_text2": detect_error_details.clinical_consequences_explain_text2,
        "clinical_consequences_flag_text2": detect_error_details.clinical_consequences_flag_text2,

        "ambiguity": detect_error_details.ambiguity
    }


    conversations_JSON_detect_results.append(detect_error_dict)

    # get the model output
    print("current model:",dspy.settings.lm.history[0]['model'],
          "\n",
          "loops:",turn_cal,
          "\n",
          "total generated results (rows):", len(conversations_JSON_detect_results))

# print_results(detect_error_details, turn_cal, conversations_JSON_detect_results)
#counter_summary_clean(Patients_results_JSON, ["errors_1", "errors_2", "errors_3"], True)
#print_results(detect_error_details, turn_cal, conversations_JSON_detect_results)

# print("current model:", dspy.settings.lm.model,
#       "\nloop times:", turn_cal,
#       "\ntotal generated results (rows):", len(conversations_JSON_detect_results))

current model: openrouter/google/gemini-2.5-flash-lite 
 loops: 1 
 total generated results (rows): 1


In [ ]:
print(dspy.settings.lm.history[0]['model'])

openrouter/openai/gpt-5-mini


current model: openrouter/openai/gpt-5-mini 
loops: 1


In [ ]:
# detect_error_details.get_lm_usage()
# dspy.inspect_history(n=1)
# dspy.inspect_history()
# setting= dspy.settings.lm
# print(setting.model)
print(dspy.settings.lm.history[2]['model'])

openrouter/openai/gpt-5-mini


In [ ]:
dspy.settings.lm.history

[{'prompt': None,
  'messages': [{'role': 'system',
    'content': "Your input fields are:\n1. `en_1` (str): Original text 1\n2. `cn_2_1` (str): Translated text 1\n3. `cn_1` (str): Original text 2\n4. `en_2_1` (str): Translated text 2\nYour output fields are:\n1. `errors_detected_text1` (str): The specific type of translation error identified from text 1/translated text 1, such as 'Units & Dosage', 'Negation', 'Terminology Error', etc.\n2. `errors_detected_domain_text1` (str): The category or domain the error falls under from text 1/translated text 1, such as 'MQM Core-Derived Errors', 'Errors in Medical Conversations', or 'English ↔ Chinese Specific Errors'.\n3. `errors_detected_domain_explain_text1` (str): Explanation why do you think it is an error.\n4. `clinical_consequences_explain_text1` (str): A brief explanation of how the error could lead to a clinical consequence.\n5. `clinical_consequences_flag_text1` (str): Flag indicating whether the error may lead to serious clinical cons

In [ ]:
conversations_JSON_detect_results_df = pd.DataFrame(conversations_JSON_detect_results)
conversations_JSON_detect_results_df.to_excel("conversations_JSON_detect_results_df.xlsx", index=False)
files.download("conversations_JSON_detect_results_df.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Error Detection validation

## Prepare dataset

In [ ]:
from google.colab import files
# folder 3. error_detected
uploaded = files.upload()

Saving conversations_JSON_detect_results_df.xlsx to conversations_JSON_detect_results_df (1).xlsx


In [ ]:
conversations_JSON_detect_results_df= pd.read_excel("conversations_JSON_detect_results_df.xlsx")
conversations_JSON_detect_results_df_JSON = conversations_JSON_detect_results_df.to_json(orient="records", force_ascii=False)
conversations_JSON_detect_results_df_JSON = json.loads(conversations_JSON_detect_results_df_JSON)

## Validating detection

In [ ]:
# 3. Define a DSPy signature
class ValidateError(dspy.Signature):
# a. Provide the instruction
    """You are an expert medical translation auditor (English ↔ Chinese).
    Your task is to evaluate whether the last translator correctly detected the translation error

You will be given:
1. Original Text: a sentence from a doctor–patient conversation.
2. Translation: the translated version.
3. Ground truth inserted error:
4. The last translator’s error judgement.

Your task:
1. Compare the original text and the translation.
Decide whether the last translator’s judgement is correct, partially correct, or incorrect.

2. Against ground truth: Compare the last translator’s judgement with the known inserted error.
Decide if it is a match, partial match, or mismatch.

3. Explain why you think the translator did well or poorly.

"""

# b. Define the input
    en_1 = dspy.InputField(desc="Original text 1")
    cn_2_1 = dspy.InputField(desc="Translated text 1") # only try this one first
    cn_1 = dspy.InputField(desc="Original text 2")
    en_2_1 = dspy.InputField(desc="Translated text 2") # only try this one first

    errors = dspy.InputField(desc="Ground truth inserted error")
    errors_domain = dspy.InputField (desc="Ground truth inserted error, which category or domain the error falls under,such as 'MQM Core-Derived Errors', 'Errors in Medical Conversations', 'English ↔ Chinese Specific Errors', or 'Identity checking errors'.")
    errors_explain = dspy.InputField(desc="Ground truth inserted error, a brief explanation of how the error could lead to a clinical consequence.")


    errors_detected_text1 = dspy.InputField(desc="The judgement of specific type of translation error identified by previous translator , from text 1/translated text 1, such as 'Units & Dosage', 'Negation', 'Terminology Error'")
    errors_detected_domain_text1  = dspy.InputField(desc="The judgement of pecific type of translation error identified by previous translator , from text 1/translated text 1, which the category or domain the error falls under, such as 'MQM Core-Derived Errors', 'Errors in Medical Conversations', 'English ↔ Chinese Specific Errors', or 'Identity checking errors'")
    errors_detected_domain_explain_text1  = dspy.InputField(desc="The judgement of why do the translator thinks it is an error from text 1/translated text 1")

    errors_detected_text2  = dspy.InputField(desc="The judgement of specific type of translation error identified by previous translator from text 2/translated text 2, such as 'Units & Dosage', 'Negation', 'Terminology Error', etc")
    errors_detected_domain_text2 = dspy.InputField(desc="The judgement of pecific type of translation error identified by previous translator from text 2/translated text 2, which the category or domain the error falls under, such as 'MQM Core-Derived Errors', 'Errors in Medical Conversations','English ↔ Chinese Specific Errors', or 'Identity checking errors'")
    errors_detected_domain_explain_text2 = dspy.InputField(desc="The judgement of why do the translator thinks it is an error from text 2/translated text 2.")
    # cn_1 = dspy.InputField(desc="Original Chinese")
    # en_2_1 = dspy.InputField(desc="Translated English")


# c. Define the output (as schema)

    translator_correct_level_text1 = dspy.OutputField(desc="Is the previous translator correct based on the original text and the translation from text 1/translated text 1? Must be exactly one of: yes | partial | no.")
    translator_correct_level_reason_text1 = dspy.OutputField(desc="Explain why the previous translator is correct, partially correct, or incorrect based only on the original text and the translation. from text 1/translated text 1")
    ground_truth_level_text1 = dspy.OutputField(desc="Does the previous translator's judgement match the ground-truth inserted error from text 1/translated text 1? Must be exactly one of: yes | partial | no.")
    ground_truth_level_reason_text1 = dspy.OutputField(desc="Explain why the previous translator's judgement is a full match, partial match, or mismatch compared to the ground-truth inserted error from text 1/translated text 1.")

    translator_correct_level_text2 = dspy.OutputField(desc="Is the previous translator correct based on the original text and the translation from text 2/translated text 2? Must be exactly one of: yes | partial | no.")
    translator_correct_level_reason_text2 = dspy.OutputField(desc="Explain why the previous translator is correct, partially correct, or incorrect based only on the original text and the translation from text 2/translated text 2.")
    ground_truth_level_text2 = dspy.OutputField(desc="Does the previous translator's judgement match the ground-truth inserted error from text 2/translated text 2? Must be exactly one of: yes | partial | no.")
    ground_truth_level_reason_text2 = dspy.OutputField(desc="Explain why the previous translator's judgement is a full match, partial match, or mismatch compared to the ground-truth inserted error from text 2/translated text 2.")


    ambiguity = dspy.OutputField(desc="Was any part of the instructions unclear? If so, please indicate what is unclear, so the author can add clarification.")


### Gemini 2.5 flash (done)

In [ ]:
# Create a model called insert_error

# follow the global setting

# validate_error = dspy.Predict(
#     signature=ValidateError
# )

In [ ]:
# set the local llm setting
lm.gm=dspy.LM(
    model="openrouter/google/gemini-2.5-flash-lite",
    api_key=api_key,
    temperature = 1,
    max_tokens=16000
)

In [ ]:
conversations_JSON_validate_results = []
model_name = None # have to do this if it is used in the context

for turn_cal, item in enumerate(conversations_JSON_detect_results[1:2], start=1):

  with dspy.context(lm=lm.gm):
    validate_error = dspy.Predict(ValidateError) # the way you can use it locally, so per each predict, it use the specified model instead of global one

    validate_error_details = validate_error(
        en_1=item['en_1'],
        cn_2_1 =item['cn_2_1'],
        cn_1=item['cn_1'],
        en_2_1 =item['en_2_1'],

        errors = item['errors'],
        errors_domain = item['errors_domain'],
        errors_explain = item['errors_explain'],

        errors_detected_text1 = item['errors_detected_text1'],
        errors_detected_domain_text1 = item['errors_detected_domain_text1'],
        errors_detected_domain_explain_text1 = item['errors_detected_domain_explain_text1'],

        errors_detected_text2 = item['errors_detected_text2'],
        errors_detected_domain_text2 = item['errors_detected_domain_text2'],
        errors_detected_domain_explain_text2 = item['errors_detected_domain_explain_text2']
        )


    validate_error_dict = {
        "case_turn_id": item['case_turn_id'],
        "setting_short": item['setting_short'],
        "speaker": item['speaker'],
        "en_1": item['en_1'],
        "cn_2_1": item['cn_2_1'],
        "errors": item['errors'],
        "errors_domain": item['errors_domain'],
        "errors_explain": item['errors_explain'],

        "errors_detected_text1": item['errors_detected_text1'],
        "errors_detected_domain_text1": item['errors_detected_domain_text1'],
        "errors_detected_domain_explain_text1": item['errors_detected_domain_explain_text1'],
        "translator_correct_level_text1": validate_error_details.translator_correct_level_text1,
        "translator_correct_level_reason_text1": validate_error_details.translator_correct_level_reason_text1,
        "ground_truth_level_text1": validate_error_details.ground_truth_level_text1,
        "ground_truth_level_reason_text1": validate_error_details.ground_truth_level_reason_text1,

        "cn_1": item['cn_1'],
        "en_2_1": item['en_2_1'],
        "errors_detected_text2": item['errors_detected_text2'],
        "errors_detected_domain_text2": item['errors_detected_domain_text2'],
        "errors_detected_domain_explain_text2": item['errors_detected_domain_explain_text2'],
        "translator_correct_level_text2": validate_error_details.translator_correct_level_text2,
        "translator_correct_level_reason_text2": validate_error_details.translator_correct_level_reason_text2,
        "ground_truth_level_text2": validate_error_details.ground_truth_level_text2,
        "ground_truth_level_reason_text2": validate_error_details.ground_truth_level_reason_text2,

        "ambiguity": validate_error_details.ambiguity
    }

    active_lm = dspy.settings.lm
    model_name = getattr(active_lm, "model", "unknown_model")
    conversations_JSON_validate_results.append(validate_error_dict)

# print("current model:",list(validate_error_details.get_lm_usage())[0],  # should match turn_cal
#       "\nloop times:", turn_cal, # number of turns processed
#       "\ntotal generated results:",len(conversations_JSON_validate_results))  # printout the model is used in the loop

In [ ]:
dspy.settings.lm.model # global model

'openrouter/openai/gpt-5-mini'

'openrouter/google/gemini-2.5-flash-lite'

In [ ]:
print_results(validate_error_details, turn_cal, conversations_JSON_validate_results)

In [ ]:
print("current model:", lm.gm.model,  # Access the model directly from lm.gm
      "\nloop times:", turn_cal, # number of turns processed
      "\ntotal generated results (rows):",len(conversations_JSON_validate_results))

current model: openrouter/google/gemini-2.5-flash-lite 
loop times: 1 
total generated results (rows): 1


In [ ]:
del validate_error_details

In [ ]:
lm.gm.history[-1]['model']

'openrouter/google/gemini-2.5-flash-lite'

In [ ]:
validate_error_details.get_lm_usage()

{}

In [ ]:
lm.history[-1]['model']

'openrouter/openai/gpt-5-mini'

In [ ]:
print("current model:", lm.gm.model,  # Access the model directly from lm.gm
      "\nloop times:", turn_cal, # number of turns processed
      "\ntotal generated results (rows):",len(conversations_JSON_validate_results))

current model: openrouter/google/gemini-2.5-flash-lite 
loop times: 1 
total generated results (rows): 1


In [ ]:
print_results(validate_error_details, turn_cal, conversations_JSON_validate_results)

IndexError: list index out of range

In [ ]:
conversations_JSON_validate_results

[{'case_turn_id': '1_1',
  'setting_short': 'Discharge',
  'speaker': 'Pharmacist',
  'en_1': 'Hello, my name’s Corey, I’m one of the pharmacists here. Can I confirm your full name and DOB? ',
  'cn_2_1': '您好，我叫 Corey，是这里的药剂师之一。请问您的名字和出生日期是？',
  'errors': 'Patients name errors (first name vs full name)',
  'errors_domain': 'Identity checking errors',
  'errors_explain': 'The source asks for the patient\'s "full name." I changed the Chinese "全名" to the less specific "名字" (name/first name). This is a plausible, subtle identity-checking error: it reduces specificity (may omit surname), could lead to mismatched records, and is clinically important when verifying identity.',
  'errors_detected_text1': 'Omission / Identity-checking error — the word “full” (全名) is omitted: “Can I confirm your full name and DOB?” → “请问您的名字和出生日期是？” (uses 名字, not 全名)',
  'errors_detected_domain_text1': 'Identity checking errors; MQM Core-Derived Errors (Undertranslation/Omission)',
  'errors_detected_domain_expl

In [ ]:
conversations_JSON_validate_results_pd = pd.DataFrame(conversations_JSON_validate_results)

In [ ]:
conversations_JSON_validate_results_pd

,case_turn_id,setting_short,speaker,en_1,cn_2_1,errors,errors_domain,errors_explain,errors_detected_text1,errors_detected_domain_text1,...,cn_1,en_2_1,errors_detected_text2,errors_detected_domain_text2,errors_detected_domain_explain_text2,translator_correct_level,translator_correct_level_reason,ground_truth_level_text2,ground_truth_level_reason_text2,ambiguity
0,1_1,Discharge,Pharmacist,"Hello, my name’s Corey, I’m one of the pharmac...",您好，我叫 Corey，是这里的药剂师之一。请问您的名字和出生日期是？,Patients name errors (first name vs full name),Identity checking errors,"The source asks for the patient's ""full name.""...",Omission / Identity-checking error — the word ...,Identity checking errors; MQM Core-Derived Err...,...,您好，我叫 Corey，是这里的药剂师之一。请问您的全名和出生日期是？,"Hello, my name’s Corey, I’m one of the pharmac...",Omission / Identity-checking error — the word ...,Identity checking errors; MQM Core-Derived Err...,The Chinese source requests 全名 (full/legal nam...,yes,"The original text `en_1` asks for ""full name a...",yes,"The ground truth error states ""Patients name e...",no
1,1_3,Discharge,Pharmacist,Thank you Kaiyu. And is now a good time to run...,谢谢你，Kaiyu。现在方便在你回家后一起复查一下你目前的所有药物吗?,Temporal,Medical Conversations — Temporal,"The phrase ""回家前"" (before you go home) was chan...","Temporal mistranslation (timing error): ""befor...",Errors in Medical Conversations (Temporal) / M...,...,谢谢你，Kaiyu。现在方便在你回家前一起复查一下你目前的所有药物吗?,Thank you Kaiyu. And is now a good time to run...,Temporal mistranslation (timing error): Chines...,Errors in Medical Conversations (Temporal) / M...,The Chinese source specifies reviewing medicat...,yes,The translator correctly identified that the C...,yes,The translator's judgement perfectly matches t...,no
2,1_5,Discharge,Pharmacist,"Great. Before we start, can I just check, do y...",很好。在我们开始之前，我想先确认一下：你对药物没有过敏，或者以前用药时没有不良反应吗？,Negation,Errors in Medical Conversations,The original question asks whether the patient...,Negation introduced / Mistranslation,MQM Core-Derived Errors (Mistranslation) / Err...,...,很好。在我们开始之前，我想先确认一下：你有没有对药物过敏，或者以前用药时有过不良反应？,"Great. Before we start, I just want to confirm...",Negation introduced / Mistranslation,MQM Core-Derived Errors (Mistranslation) / Err...,The back-translation renders the Chinese quest...,yes,The translator correctly identified that the b...,yes,"The translator's detected error (""Negation int...",No ambiguity.
3,1_7,Discharge,Pharmacist,Okay. The doctor has prescribed you an antibio...,好的。医生给你开了一种名为阿莫西林的抗生素，常见的商品名是 Augmentin Duo Fo...,Omission,MQM Core-Derived Errors,"I removed ""clavulanic acid"" (克拉维酸) from the dr...","Omission / Terminology Error — the phrase ""wit...",MQM Core-Derived Errors,...,好的。医生给你开了一种名为阿莫西林配克拉维酸的抗生素，常见的商品名是 Augmentin D...,Okay. The doctor has prescribed you an antibio...,"Omission / Terminology Error — ""配克拉维酸"" (with c...",MQM Core-Derived Errors,"The Chinese source names the combination ""阿莫西林...",yes,"The original text 2 correctly states ""amoxicil...",no,"The ground truth error states that ""clavulanic...",no
4,1_8,Discharge,Pharmacist,The dose is one tablet every morning and night...,剂量是早晚各一片，共服用 7 周。最好随餐服用，这样有助于吸收，也能减少胃部不适。比如，你可...,Units & Dosage,Errors in Medical Conversations,"The duration ""7 天"" (7 days) was changed to ""7 ...",Units & Dosage & Temporal (duration mistransla...,Errors in Medical Conversations; MQM Core-Deri...,...,剂量是早晚各一片，共服用 7 天。最好随餐服用，这样有助于吸收，也能减少胃部不适。比如，你可...,The dose is one tablet every morning and night...,Units & Dosage & Temporal (duration mistransla...,Errors in Medical Conversations; MQM Core-Deri...,"The Chinese source specifies ""共服用 7 天"" (7 days...",yes,The translator correctly identified that the d...,yes,The ground truth error states that the duratio...,no
5,1_9,Discharge,Pharmacist,"Most people don’t have any major side-effects,...",大多数人会出现严重的副作用，但有些人可能会出现大便稀（或腹泻）、轻微的恶心或胃部不适。不过这...,Negation,Errors in Medical Conversations - Negation,"The negative particle ""不会"" (will not / don't) ...",Negation / Mistranslation (polarity 

### Gemini-2.5-flash validation (not done)

In [ ]:
# 1. Define the language model
lm_gemini=dspy.LM(
    model="openrouter/google/gemini-2.5-flash-lite",
    api_key=api_key,
    temperature = 1,
    max_tokens=16000
)

In [ ]:
validate_error = dspy.Predict(
    signature=DetectError,
    # lm=dspy.LM(
    # model="openrouter/google/gemini-2.5-flash-lite",
    # api_key=api_key,
    # temperature = 1,
    # max_tokens=16000
    # )
     lm=dspy.LM(lm_gemini
)
     )

In [ ]:
conversations_JSON_validate_results_gemini = []

for turn_cal, item in enumerate(conversations_JSON_detect_results[1:2], start=1):
    validate_error_details = validate_error(
        en_1=item['en_1'],
        cn_2_1 =item['cn_2_1'],
        errors = item['errors'],
        errors_domain = item['errors_domain'],
        errors_explain = item['errors_explain'],
        errors_detected = item['errors_detected'],
        errors_detected_domain = item['errors_detected_domain'],
        errors_detected_domain_explain = item['errors_detected_domain_explain']
        )

    validate_error_dict = {
        "case_turn_id": item['case_turn_id'],
        "setting_short": item['setting_short'],
        "speaker": item['speaker'],
        "en_1": item['en_1'],
        "cn_2_1": item['cn_2_1'],
        "errors": item['errors'],
        "errors_domain": item['errors_domain'],
        "errors_explain": item['errors_explain'],
        "errors_detected": item['errors_detected'],
        "errors_detected_domain": item['errors_detected_domain'],
        "errors_detected_domain_explain": item['errors_detected_domain_explain'],
        "translator_correct_level": validate_error_details.translator_correct_level,
        "ground_truth_level": validate_error_details.ground_truth_level,
        "translator_explain": validate_error_details.translator_explain,
        "ambiguity": validate_error_details.ambiguity
    }
    conversations_JSON_validate_results_gemini .append(validate_error_dict)

print(turn_cal)       # number of turns processed
print(len(conversations_JSON_validate_results_gemini ))  # should match turn_cal

In [ ]:
Counter(
    validation_result
    for item in conversations_JSON_validate_results_gemini
    for validation_result in (item.get("translator_correct_level"), item.get("ground_truth_level"))
)

### Deepseek/deepseek-r1-0528-qwen3-8b (no done)

In [ ]:
validate_error = dspy.Predict(
    signature=DetectError,
    lm=dspy.LM(
    model="openrouter/deepseek/deepseek-r1-0528-qwen3-8b",
    api_key=api_key,
    temperature = 1,
    max_tokens=16000
    )
)

In [ ]:
dspy.Predict(InsertError) #normal prediction

In [ ]:
conversations_JSON_validate_results_deepseek = []

for turn_cal, item in enumerate(conversations_JSON_detect_results, start=1):
    validate_error_details = validate_error(
        en_1=item['en_1'],
        cn_2_1 =item['cn_2_1'],
        errors = item['errors'],
        errors_domain = item['errors_domain'],
        errors_explain = item['errors_explain'],
        errors_detected = item['errors_detected'],
        errors_detected_domain = item['errors_detected_domain'],
        errors_detected_domain_explain = item['errors_detected_domain_explain']
        )

    validate_error_dict = {
        "case_turn_id": item['case_turn_id'],
        "setting_short": item['setting_short'],
        "speaker": item['speaker'],
        "en_1": item['en_1'],
        "cn_2_1": item['cn_2_1'],
        "errors": item['errors'],
        "errors_domain": item['errors_domain'],
        "errors_explain": item['errors_explain'],
        "errors_detected": item['errors_detected'],
        "errors_detected_domain": item['errors_detected_domain'],
        "errors_detected_domain_explain": item['errors_detected_domain_explain'],
        "translator_correct_level": validate_error_details.translator_correct_level,
        "ground_truth_level": validate_error_details.ground_truth_level,
        "translator_explain": validate_error_details.translator_explain,
        "ambiguity": validate_error_details.ambiguity
    }

    conversations_JSON_validate_results_deepseek.append(validate_error_dict)

print(turn_cal)       # number of turns processed
print(len(conversations_JSON_validate_results_deepseek))  # should match turn_cal

In [ ]:
Counter(
    validation_result
    for item in conversations_JSON_validate_results_deepseek
    for validation_result in (item.get("translator_correct_level"), item.get("ground_truth_level"))
)

In [ ]:
[
    {
        "id": item.get("case_turn_id"),
        "en_1": item.get("en_1"),
        "cn_2_1": item.get("cn_2_1"),
        "errors_domain": item.get("errors_domain"),
        "errors_detected": item.get("errors_detected"),
        "translator_explain": item.get("translator_explain"),
        "translator_correct_level": item.get("translator_correct_level"),
        "ground_truth_level": item.get("ground_truth_level")
    }
    for item in conversations_JSON_validate_results_deepseek
    if (
        item.get("translator_correct_level") == "partial"
        or item.get("ground_truth_level") == "partial"
    )
]

In [ ]:
ds_results = Counter(
    validation_result
    for item in conversations_JSON_validate_results_deepseek
    for validation_result in (item.get("translator_correct_level"), item.get("ground_truth_level"))
)

gm_results = Counter(
     validation_result
    for item in conversations_JSON_validate_results_gemini
    for validation_result in (item.get("translator_correct_level"), item.get("ground_truth_level")))

gpt_results = Counter(
     validation_result
    for item in conversations_JSON_validate_results
    for validation_result in (item.get("translator_correct_level"), item.get("ground_truth_level")))




In [ ]:
print(
    f"ds:  {ds_results}\n"
    f"gm:  {gm_results}\n"
    f"gpt: {gpt_results}")


# Combine all the tables and manually check them

In [ ]:
conversations_JSON_validate_results_gemini_df = pd.DataFrame(conversations_JSON_validate_results_gemini)
conversations_JSON_validate_results_deepseek_df = pd.DataFrame(conversations_JSON_validate_results_deepseek)
conversations_JSON_validate_results_df = pd.DataFrame(conversations_JSON_validate_results)
conversations_JSON_df = pd.DataFrame(conversations_JSON)

In [ ]:
conversations_JSON_validate_results_gemini_df = conversations_JSON_validate_results_gemini_df.rename(
    columns={
    "translator_correct_level": "translator_correct_level_translator_correct_gemini",
    "ground_truth_level": "ground_truth_level_gemini",
    "translator_explain": "translator_explain_gemini"
})
conversations_JSON_validate_results_df = conversations_JSON_validate_results_df.rename(
    columns={
    "translator_correct_level": "translator_correct_level_translator_correct_gpt",
    "ground_truth_level": "ground_truth_level_gpt",
    "translator_explain": "translator_explain_gpt"
})
conversations_JSON_validate_results_deepseek_df = conversations_JSON_validate_results_deepseek_df.rename(
    columns={
    "translator_correct_level": "translator_correct_level_translator_correct_deepseek",
    "ground_truth_level": "ground_truth_level_deepseek",
    "translator_explain": "translator_explain_deepseek"
})


In [ ]:
gemini_small = conversations_JSON_validate_results_gemini_df[["case_turn_id",
                                                              "translator_correct_level_translator_correct_gemini",
                                                               "ground_truth_level_gemini",
                                                              "translator_explain_gemini"]]

deepseek_small = conversations_JSON_validate_results_deepseek_df[["case_turn_id",
                                                              "translator_correct_level_translator_correct_deepseek",
                                                               "ground_truth_level_deepseek",
                                                                  "translator_explain_deepseek"]]

conversations_JSON_df_small = conversations_JSON_df[["case_turn_id",
                                                     "cn_1",
                                                     "en_2_1"]]


In [ ]:
validation_comparison = conversations_JSON_validate_results_df.merge(conversations_JSON_df_small, on="case_turn_id").merge(gemini_small, on="case_turn_id").merge(deepseek_small, on="case_turn_id")

In [ ]:
validation_comparison = validation_comparison[
    [
        'case_turn_id', 'setting_short', 'speaker',
        'en_1', 'cn_1', 'en_2_1', 'cn_2_1',
        'errors', 'errors_domain', 'errors_explain',
        'errors_detected', 'errors_detected_domain',
        'translator_correct_level_translator_correct_gpt',
        'ground_truth_level_gpt', 'translator_explain_gpt',
        'ambiguity',
        'translator_correct_level_translator_correct_gemini',
        'ground_truth_level_gemini', 'translator_explain_gemini',
        'translator_correct_level_translator_correct_deepseek',
        'ground_truth_level_deepseek', 'translator_explain_deepseek'
    ]
]


In [ ]:
from google.colab import files

validation_comparison.to_excel("validation_comparison.xlsx", index=False)
files.download("validation_comparison.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>